In [14]:
import os
import django
import pandas as pd
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fxfutbol.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [15]:
import ssl
import http.client
import json
import pandas as pd

class SofaScoreAPI:
    def __init__(self, match_id):
        self.match_id = match_id
        self.base_url = 'www.sofascore.com'

    def _make_request(self, endpoint):
        """
        Belirtilen endpoint'e HTTP isteği yapar ve yanıtı döner.
        """
        context = ssl._create_unverified_context()
        con = http.client.HTTPSConnection(self.base_url, context=context)
        try:
            con.request("GET", endpoint)
            response = con.getresponse()
            data = response.read()
            data = json.loads(data)
            return {"match_id":self.match_id,"response_code": response.status, "data": data}
        except Exception as e:
            con.request("GET", endpoint)
            response = con.getresponse()
            return {"match_id":self.match_id,"response_code": response.status , "data": None }

    def get_lineups(self):
        """
        Kadro bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/lineups'
        return self._make_request(endpoint)

    def get_incidents(self):
        """
        Maç olaylarını döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/incidents'
        return self._make_request(endpoint)

    def get_graph(self):
        """
        Maç baskı grafiğini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/graph'
        return self._make_request(endpoint)

    def get_match_info(self):
        """
        Maç bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}'
        return self._make_request(endpoint)

    def get_odds(self):
        """
        Maç oran bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/odds/672/featured'
        return self._make_request(endpoint)

    def get_pregame_form(self):
        """
        Takım form bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/pregame-form'
        return self._make_request(endpoint)

    def get_statistics(self):
        """
        Maç istatistiklerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/statistics'
        return self._make_request(endpoint)

    def fetch_all_data(self):
        """
        Tüm API endpoint'lerinden verileri toplar ve bir dict olarak döner.
        """
        return {
            "match_id": self.match_id,
            "lineups": self.get_lineups(),
            "incidents": self.get_incidents(),
            "graph": self.get_graph(),
            "match_info": self.get_match_info(),
            "odds": self.get_odds(),
            "pregame_form": self.get_pregame_form(),
            "statistics": self.get_statistics(),
        }
        
match_id = 12531328
sofa_api = SofaScoreAPI(match_id)
result_all = sofa_api.fetch_all_data()
result_all

{'match_id': 12531328,
 'lineups': {'match_id': 12531328,
  'response_code': 404,
  'data': {'error': {'code': 404, 'message': 'Not Found'}}},
 'incidents': {'match_id': 12531328,
  'response_code': 200,
  'data': {'incidents': [{'text': 'FT',
     'homeScore': 3,
     'awayScore': 2,
     'isLive': False,
     'time': 90,
     'addedTime': 999,
     'timeSeconds': 5400,
     'reversedPeriodTime': 1,
     'reversedPeriodTimeSeconds': 0,
     'incidentType': 'period'},
    {'from': 'regular',
     'homeScore': 3,
     'awayScore': 2,
     'playerName': 'Cynthia Adamu',
     'period': 'period2',
     'id': 188019436,
     'time': 80,
     'isHome': True,
     'incidentClass': 'regular',
     'reversedPeriodTime': 11,
     'incidentType': 'goal'},
    {'from': 'regular',
     'homeScore': 2,
     'awayScore': 2,
     'playerName': 'Lena Kovar ',
     'period': 'period2',
     'id': 188019237,
     'time': 78,
     'isHome': False,
     'incidentClass': 'regular',
     'reversedPeriodTime'

In [17]:
import http.client
import pandas as pd
import json
from datetime import datetime, timedelta
from analysis.models import MatchStats  # `MatchStats` modelini içe aktarın


class ResultAPI:
    def __init__(self, date="2023-01-01"):
        self.date = date
        self.full_df = pd.DataFrame()

    def AddDataBase(self):

            df = self.gunluk_fikstur()
            MatchStats.objects.filter(id__in=df['id'].tolist()).delete()
            
            fixture_list = [
                MatchStats(
                    id=row['id'],  # ID manuel olarak girilecek
                    tarih=row['tarih'],  # Tarih alanı ekleniyor
                    stats=row['Stats']  # JSON formatındaki veriyi 'stats' alanına ekle
                )
                for index, row in df.iterrows()
            ]
            
            MatchStats.objects.bulk_create(fixture_list)
            print("Mevcut veriler silindi ve yeni veriler eklendi.")
        
    def gunluk_fikstur(self):
        try:
            conn = http.client.HTTPSConnection('www.sofascore.com')
            conn.request(
                'GET',
                f'/api/v1/sport/football/scheduled-events/{self.date}'
            )
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)["events"]

            model_data = [{
                'tarih': self.date,
                'data': data,
              
            }]
            
            df = pd.DataFrame(model_data)
        except Exception as e:
            model_data = [{
                'tarih': self.date,
                'data': [],
                
            }]
        try:
            
            df = pd.DataFrame(model_data)
            df_exploded = df.explode('data')
            expanded_data = pd.json_normalize(df_exploded['data'])
            expanded_data['tarih'] = list(df_exploded['tarih'])
            data = expanded_data.drop_duplicates(subset='id', keep='last')
            data = data[data["status.type"] == "finished"]
            
            
        except:
            pass
        
   
        data_stat = []
        toplam = len(data)
        for idx in list(data["id"]):
            toplam = toplam-1
            
            try:
               
                sofa_api = SofaScoreAPI(idx)
                result_all = sofa_api.fetch_all_data()
                data_stat.append(result_all)
                print("{} işlem başarılı kalan işlem sayısı".format(toplam))
            except:
                print("{} işlem ERRORRRR kalan işlem sayısı".format(toplam))
                result_all = {}
                data_stat.append(result_all)
                
                
        data["Stats"] = data_stat

        columns = ['id','tarih','Stats']
        result_data = data[columns]

        return result_data
    
api = ResultAPI(date = "2024-10-07")
data  = api.AddDataBase()
data

311 işlem başarılı kalan işlem sayısı
310 işlem başarılı kalan işlem sayısı
309 işlem başarılı kalan işlem sayısı
308 işlem başarılı kalan işlem sayısı
307 işlem başarılı kalan işlem sayısı
306 işlem başarılı kalan işlem sayısı
305 işlem başarılı kalan işlem sayısı
304 işlem başarılı kalan işlem sayısı
303 işlem başarılı kalan işlem sayısı
302 işlem başarılı kalan işlem sayısı
301 işlem başarılı kalan işlem sayısı
300 işlem başarılı kalan işlem sayısı
299 işlem başarılı kalan işlem sayısı
298 işlem başarılı kalan işlem sayısı
297 işlem başarılı kalan işlem sayısı
296 işlem başarılı kalan işlem sayısı
295 işlem başarılı kalan işlem sayısı
294 işlem başarılı kalan işlem sayısı
293 işlem başarılı kalan işlem sayısı
292 işlem başarılı kalan işlem sayısı
291 işlem başarılı kalan işlem sayısı
290 işlem başarılı kalan işlem sayısı
289 işlem başarılı kalan işlem sayısı
288 işlem başarılı kalan işlem sayısı
287 işlem başarılı kalan işlem sayısı
286 işlem başarılı kalan işlem sayısı
285 işlem ba